## <u>Human-in-the-loop</u>

Provide human feedback to a team.

Two main ways to achieve this
1. During .run/.run_stream - provide feedback through UserProxyAgent.
2. Once the run terminates, provide feedback through .run/.run_stream

#### <u>Off topic : code samples on integration with web and UI frameworks</u>

1. <u>[Agentchat + FastAPI](https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_fastapi)</u>
2. <u>[Agentchat + ChainLit](https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_chainlit)</u>
3. <u>[Agentchat + StreamLit](https://github.com/microsoft/autogen/tree/main/python/samples/agentchat_streamlit)</u>

### <u>Providing feedback during the run</u>

As the name goes **UserProxyAgent** is a built-in agent that acts as a proxy for the user and provides feedback.

Create an instance of UPA and include it in the team definition. The team will decide when to call UPA to ask for feedback.

In **RoundRobinGroupChat**, UPA is called by order but in **SelectorGroupChat**, the selector prompt or selector function determines when the UPA is called.

<img src="images/image_14.png" width=600>

Task & TaskResult arrows signify the normal input of task (run/run_stream) and the output TaskResult. The bold arrows show us the flow when the group chat decides to take an opinion from the user and then use that to continue finding the solution.

<u>**Note:**</u> When the UPA is called, until a human replies, the state is suspended mid-execution. If the system crashes at that time, there is no way for the team to resume later.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)

termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

await Console(team.run_stream(task="Write a 4 line poem about Pune in Marathi."))
await model_client.close()

---------- TextMessage (user) ----------
Write a 4 line poem about Pune in Marathi.
---------- TextMessage (assistant) ----------
पुणे शहर, संस्कृतीची गळामिठी,  
चहा-कॉफीचं, भेटींचं गाणं गाती,  
सार्वजनिक उद्याने, सुखद सावली,  
पुण्याची गोष्ट, हृदयात वसती.  

TERMINATE


Enter your response:  do again


---------- TextMessage (user_promy) ----------
do again
---------- TextMessage (assistant) ----------
पुणे म्हातारं, ज्ञानाचं घर,  
पार्वती पर्वता, निसर्गात मजर,  
छान गप्पा, चहा-भाजी,  
पुण्याची माती, प्रेमभरी सावली.  

TERMINATE


Enter your response:  APPROVE


---------- TextMessage (user_promy) ----------
APPROVE


### <u>Providing feedback to the next run</u>

In the last example, using UPA we were providing feedback within the same run.

Now we will provide feedback within each run.

This process runs best in a following format. A team completes a run. The state is saved in persistent storage. Then the team turns to the user input and then resume the run again when feedback arrives.

There are two ways to implement this approach:
1. Set max number : team always stops after # of runs.
2. Use termination condition to give control back.

<img src="images/image_15.png" width=500>

#### <u>Using max turns</u>

In the below example, following pattern is shown. Chatbot pattern where the team terminates and gets user input after every 1 turn.

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Write 4 lines of information about a vacation destination in India."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break
await model_client.close()

---------- TextMessage (user) ----------
Write 4 lines of information about a vacation destination in India.
---------- TextMessage (assistant) ----------
Rajasthan is a popular vacation destination in India, known for its rich history and vibrant culture. Visitors can explore majestic forts and palaces, such as the mesmerizing Amer Fort in Jaipur and the stunning City Palace. The Thar Desert offers unique experiences like camel safaris and traditional Rajasthani cuisine. Additionally, the annual Pushkar Camel Fair attracts tourists with its colorful festivities and cultural events.


Enter your feedback (type 'exit' to leave):  now a different one


---------- TextMessage (user) ----------
now a different one
---------- TextMessage (assistant) ----------
Goa is a renowned vacation destination in India, famous for its beautiful beaches and vibrant nightlife. With its charming Portuguese-influenced architecture, visitors can explore historic churches and lively markets. Goa offers a variety of water sports, including parasailing and scuba diving, making it perfect for adventure seekers. The laid-back atmosphere, combined with delicious seafood and beach shacks, makes it a haven for relaxation and enjoyment.


Enter your feedback (type 'exit' to leave):  exit


#### <u>Using termination condition (handofftermination)</u>

In this example we will look at handoff termination.

Here is what happens in HandOff termination

<img src="images/image_16.png" width=700>

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="If you cannot complete the task, transfer to user. Otherwise, when finished, respond with 'TERMINATE'.",
)

# Define a termination condition that checks for handoff messages.
handoff_termination = HandoffTermination(target="user")
# Define a termination condition that checks for a specific text mention.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team with the lazy assistant and both termination conditions.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=handoff_termination | text_termination)

# Run the team and stream to the console.
task = "What is the weather in Pune right now?"
await Console(lazy_agent_team.run_stream(task=task), output_stats=True)

---------- TextMessage (user) ----------
What is the weather in Pune right now?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_TVZNt4xWWv5PkN3rQNoYQbey', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 70, Completion tokens: 11]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_TVZNt4xWWv5PkN3rQNoYQbey', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 70
Total completion tokens: 11
Duration: 1.86 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 36, 11, 68824, tzinfo=datetime.timezone.utc), content='What is the weather in Pune right now?', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=11), metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 36, 12, 849323, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_TVZNt4xWWv5PkN3rQNoYQbey', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 36, 12, 849323, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_TVZNt4xWWv5PkN3rQNoYQbey', is_error=False)], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=

<u>You see above that, since we have configured the team/agent using the HandOff and the HandOff condition was that "if the agent does not know how to do a task, he will hand it off to the user" so that is what he did.</u>

Now, let us resume the team's run i.e. start a new run with a user input.

In [4]:
await Console(lazy_agent_team.run_stream(task="The weather in Pune is sunny."))

---------- TextMessage (user) ----------
The weather in Pune is sunny.
---------- TextMessage (lazy_assistant) ----------
Thank you for the update! If you need any further information or assistance, feel free to ask.
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_SZUV3iQBmshpM2YDT4JKUVar', arguments='{}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_SZUV3iQBmshpM2YDT4JKUVar', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 37, 58, 396754, tzinfo=datetime.timezone.utc), content='The weather in Pune is sunny.', type='TextMessage'), TextMessage(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=106, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 38, 0, 539149, tzinfo=datetime.timezone.utc), content='Thank you for the update! If you need any further information or assistance, feel free to ask.', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=130, completion_tokens=11), metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 38, 3, 54090, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_SZUV3iQBmshpM2YDT4JKUVar', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, metadata=

<u>Interestingly, it used the handoff again. So now it terminated the run and is waiting for the user.</u>

In [5]:
await Console(lazy_agent_team.run_stream(task="terminate the session."))

---------- TextMessage (user) ----------
terminate the session.
---------- TextMessage (lazy_assistant) ----------
TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 40, 0, 209910, tzinfo=datetime.timezone.utc), content='terminate the session.', type='TextMessage'), TextMessage(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=163, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 16, 18, 40, 1, 11451, tzinfo=datetime.timezone.utc), content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")